In [ ]:
import math

In [ ]:
import numpy as np

In [ ]:
!pip install galois
!pip install reedsolo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import galois
from reedsolo import RSCodec

In [ ]:
class Decoding:

  def DNA_to_Bi(arr):
    bi_arr = np.zeros((np.size(arr, axis = 0), (np.size(arr, axis = 1)*2)), dtype = np.int64)
    for i in range(0, np.size(arr, axis = 0)):
      for j in range(0, np.size(arr, axis = 1)):
        if arr[i, j] == 'G':
          bi_arr[i, 2*j] = 0
          bi_arr[i, 2*j+1] = 0
        elif arr[i, j] == 'A':
          bi_arr[i, 2*j] = 1
          bi_arr[i, 2*j+1] = 0
        elif arr[i, j] == 'C':
          bi_arr[i, 2*j] = 0
          bi_arr[i, 2*j+1] = 1
        elif arr[i, j] == 'T':
          bi_arr[i, 2*j] = 1
          bi_arr[i, 2*j+1] = 1
        else:
          print("값이 ATCG가 아님 :", arr[i, j])
          raise ValueError
    return bi_arr


  def bi_to_deci(arr):
    sum = 0
    pos = 0
    for i in range(0, len(arr)):
      sum += arr[i] * 2 ** pos
      pos += 1
    return sum

  def pari_to_deci(arr):

    parity_deci = np.zeros((np.size(arr, axis = 0), 8),dtype = np.int64)
    pos = 0
    for i in range(0, np.size(arr, axis = 0)):
      sum = 0
      j = 0
      while(1):
        if j == 64:
          break
        sum += arr[i, j] * 2 ** pos
        j+=1
        pos+=1
        if j % 8 == 0:
          parity_deci[i, (j//8)-1] = sum
          sum = 0 
          pos = 0
      return parity_deci

  def check_reed(arr, reed):
    #리드솔로몬 만들었을때 나온 array
    arr_index_pari= np.concatenate((arr, reed), axis = 1).astype('uint8')
    parity = 8
    gf_en = galois.GF(2**3)#이건 그냥 2**3으로 한다고 약속 하고 한거
    rsc = RSCodec(parity)#8개의 parity를 만든다.
    #rsc 디코딩한 값과 다르면 디코딩한 데이터로 치환
    for i in range(0, np.size(arr, axis = 0)):
      if np.all((np.array(rsc.decode(arr_index_pari[i])[0]) == arr[i])):
        print("리드솔로몬결과 데이터값 이상 없음")
        pass
      else:
        arr[i] = np.array(rsc.decode(arr_index_pari[i])[0])
        print(i, ", 에서 바꿔짐")
      return arr
    

In [ ]:
DNA_data = []
text_file = open('/content/drive/MyDrive/ascii/uni_english.txt', 'r')
while True:
    line = text_file.readline()
    if not line: break
    DNA_data.append(line[:-1])
text_file.close()

In [ ]:
DNA_data

['TTCGTTCGTCGTTGATTGGTATGCCGTGTCCGGAAGCAATACGGCGCTCGCTCCACGGCAGATAGTTCATGCTATACTCTCAACTCTTCACGGCTCGTATGAGCCCTTCTGGGGTGCAAAAAGCGGCATGAAACGGAGCCATGAGTTTGT',
 'TTCGTTCGTCGTTGATTGGTTTGCTCTGCACGGCTGACCATCCATGGTGCTTAGTGCACGGCGCTCGCTCCACGGCGAATAGTCGACTATCCACTCTTAATCTTTGCCCTAAATCAAGCCTGTTACGAGGAAACGGAGCCATGAGTTTGT',
 'TTCGTTCGTCGTTGATTGGTGGCCGTCGGCCGACTCTGCACGGCGCTCGCTCCACGGCCCATGGTAGTCAATGAACTCCCTGGCCGAGTGTCCGGAGTGACGTCTAAACCAAAGTCTCTCCATCCGCAAGAAACGGAGCCATGAGTTTGT',
 'TTCGTTCGTCGTTGATTGGTCGCCCATTATTATGGAGAATGAATAGTGGAAAGTCGACCATGCTCGCTGGAATATCGATCATGGAGTGTTTACCATCTCGCACTTAGAGTTTGACGATATTTTAGCATTGAAACGGAGCCATGAGTTTGT',
 'TTCGTTCGTCGTTGATTGGTAGCCACACTCCATCATTGTCGGCCGACTCTGCACGGCCCATACTGGTCAATCATCACCGGAGAGTCGACCATTAACTCTCAAATTTCAGGGCTCCCCTATTCGGGAATGGAAACGGAGCCATGAGTTTGT',
 'TTCGTTCGTCGTTGATTGGTTGCCTCCATCATTGTCGGCTCTTGGTTGTCGGCTTACTATCCACCATGGAGAATGAATAGTGGAACATCCTCCATACATAAAAAACCCGCACAAATATATTGGCAGTCCTAAACGGAGCCATGAGTTTGT',
 'TTCGTTCGTCGTTGATTGGTGCCCTGGTTGATAGTGGAATCTACACGGCGCTCGCTCCACGGCGCATA

In [ ]:
len(DNA_data[0])

150

In [ ]:
payload = []
for seq in DNA_data:
  payload.append(seq[20:-20])

In [ ]:
payload

['ATGCCGTGTCCGGAAGCAATACGGCGCTCGCTCCACGGCAGATAGTTCATGCTATACTCTCAACTCTTCACGGCTCGTATGAGCCCTTCTGGGGTGCAAAAAGCGGCATG',
 'TTGCTCTGCACGGCTGACCATCCATGGTGCTTAGTGCACGGCGCTCGCTCCACGGCGAATAGTCGACTATCCACTCTTAATCTTTGCCCTAAATCAAGCCTGTTACGAGG',
 'GGCCGTCGGCCGACTCTGCACGGCGCTCGCTCCACGGCCCATGGTAGTCAATGAACTCCCTGGCCGAGTGTCCGGAGTGACGTCTAAACCAAAGTCTCTCCATCCGCAAG',
 'CGCCCATTATTATGGAGAATGAATAGTGGAAAGTCGACCATGCTCGCTGGAATATCGATCATGGAGTGTTTACCATCTCGCACTTAGAGTTTGACGATATTTTAGCATTG',
 'AGCCACACTCCATCATTGTCGGCCGACTCTGCACGGCCCATACTGGTCAATCATCACCGGAGAGTCGACCATTAACTCTCAAATTTCAGGGCTCCCCTATTCGGGAATGG',
 'TGCCTCCATCATTGTCGGCTCTTGGTTGTCGGCTTACTATCCACCATGGAGAATGAATAGTGGAACATCCTCCATACATAAAAAACCCGCACAAATATATTGGCAGTCCT',
 'GCCCTGGTTGATAGTGGAATCTACACGGCGCTCGCTCCACGGCGCATAGTCCACGATGTAGGGCCGACTCTGCACGGCGCGCATAAAGATTATCTGGTCTCGTATATCCG',
 'CCCCGCTCGCTCCACGGCTGCCGATCAACCATCAACAATGGAATCTACACGGCTCGTTCTGCACGGCTCTTGGTTGTCCTGTAGTCGACCGACCTCTATACTCTCGTTGT',
 'ACCCGGCGAATTCTACTTAGTAGTTGCTATATTGTGGAATCTACTTAGTAGTCGGCGCTCGCTCCACGGCTCTTGGTG

In [ ]:
bi_arr = np.zeros((len(payload), len(payload[0])*2), dtype = np.int64)
for i in range(0, len(payload)):
  for j in range(0, len(payload[0])):
    if payload[i][j] == 'G':
      bi_arr[i][2*j] = 0
      bi_arr[i][2*j+1] = 0
    elif payload[i][j] == 'C':
      bi_arr[i][2*j] = 1
      bi_arr[i][2*j+1] = 0
    elif payload[i][j] == 'A':
      bi_arr[i][2*j] = 0
      bi_arr[i][2*j+1] = 1
    elif payload[i][j] == 'T':
      bi_arr[i][2*j] = 1
      bi_arr[i][2*j+1] = 1
    else:
      print("값이 ATCG가 아님 :", payload[j])
      raise ValueError

In [ ]:
bi_arr.shape

(100, 220)

In [ ]:
bi_arr[0]

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0])

In [ ]:
index_bits = np.size(bi_arr, axis=0)
index_bits = math.ceil(math.log2(index_bits))
if index_bits % 2 == 1:
  index_bits+=1
print(index_bits)

8


In [ ]:
index_arr = bi_arr[:,:index_bits]
mess_arr = bi_arr[:,index_bits:-64]
reed_arr = bi_arr[:, -64:]

In [ ]:
index_arr

array([[0, 1, 1, 1, 0, 0, 1, 0],
       [1, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 1, 0, 1, 0, 1, 0],
       [1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 1, 0],
       [1, 0, 0, 1, 1, 0, 1, 0],
       [0, 1, 0, 1, 1, 0, 1, 0],
       [1, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 1, 1, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 1, 1, 0, 1, 0],
       [1, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 1, 1, 0],
       [1, 0, 1, 0, 0, 1, 1, 0],
       [0, 1, 1, 0, 0, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 1, 1, 0],
       [1, 1, 0, 1, 0, 1, 1, 0],
       [0,

In [ ]:
index = []
for i in range(0, np.size(index_arr, axis = 0)):
  index.append(Decoding.bi_to_deci(index_arr[i]))

In [ ]:
len(index)

100

In [ ]:
deci_index = index-index[0]
print(deci_index)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [ ]:
lang = Decoding.bi_to_deci(mess_arr[-1][0:6])
txt_Null = Decoding.bi_to_deci(mess_arr[-1][6:14])
fragments = Decoding.bi_to_deci(mess_arr[-1][14:22])
index_bits_len = Decoding.bi_to_deci(mess_arr[-1][22:30])

In [ ]:
print(lang , txt_Null, fragments, index_bits_len)

1 92 100 8


In [ ]:
mess = mess_arr[0:-1].flatten()
mess = mess[:-txt_Null]

In [ ]:
mess.size

14560

In [ ]:
mess = mess.reshape((mess.size//7, 7))

In [ ]:
mess

array([[1, 0, 0, ..., 1, 1, 0],
       [0, 1, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 1, 1, ..., 0, 1, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
ascii = []
for i in range(0, np.size(mess, axis = 0)):
  ascii.append(Decoding.bi_to_deci(mess[i]))

In [ ]:
texts = []
for i in range(0, len(ascii)):
  texts.append(chr(ascii[i]))

In [ ]:
tmp_DNA = "".join(texts)

In [ ]:
tmp_DNA

'1. In the beginning God created the heavens and the earth.\r\n2. Now the earth was formless and empty, darkness was over the surface of the deep, and the Spirit of God was hovering over the waters.\r\n3. And God said, "Let there be light," and there was light.\r\n4. God saw that the light was good, and He separated the light from the darkness.\r\n5. God called the light "day," and the darkness he called "night." And there was evening, and there was morning--the first day.\r\n \r\n \t\r\n6. And God said, "Let there be an expanse between the waters to separate water from water."\r\n7. So God made the expanse and separated the water under the expanse from the water above it. And it was so.\r\n8. God called the expanse "sky." And there was evening, and there was morning--the second day.\r\n9. And God said, "Let the water under the sky be gathered to one place, and let dry ground appear." And it was so.\r\n10. God called the dry ground "land," and the gathered waters he called "seas." And 

In [ ]:
writing = input("Decoded txt write(y/n) : ")
if writing == 'y' or 'Y':
#  pd.DataFrame(DNA_library).to_csv(path_or_buf = '/content/drive/MyDrive/Bible/'+'Bible'+"_"+'150bp'+"_seq.csv", index = False)
#  pd.DataFrame(DNA_library).to_csv(path_or_buf = '/content/drive/MyDrive/Bible/'+'Bible'+"_"+'150bp'+"_seq.txt", index = False,  columns= None)
  f = open('/content/drive/MyDrive/ascii/decode.txt', 'w')
  f.write(tmp_DNA)
  f.close()

DNA write(y/n) : y


In [ ]:
#ascii code는 7자리를 사용함

In [ ]:
#단어 한개짜리로 했을때 쓴거
'''
if bi_arr.size % 7 != 0:
  bi_arr = bi_arr[:bi_arr.size - 1]
else:
  pass
bi_arr = bi_arr.reshape(bi_arr.size//7, 7)
ascii = []
for i in range(0, np.size(bi_arr, axis = 0)):
  ascii.append(Decoding.bi_to_deci(bi_arr[i]))
for i in range(0, len(ascii)):
  ascii[i] = chr(ascii[i])
parity = ascii[-2:]
ascii = ascii[:-2]
ascii = ''.join(ascii)
'''

In [ ]:
writing = input("DNA write(y/n) : ")
if writing == 'y' or 'Y':
#  pd.DataFrame(DNA_library).to_csv(path_or_buf = '/content/drive/MyDrive/Bible/'+'Bible'+"_"+'150bp'+"_seq.csv", index = False)
#  pd.DataFrame(DNA_library).to_csv(path_or_buf = '/content/drive/MyDrive/Bible/'+'Bible'+"_"+'150bp'+"_seq.txt", index = False,  columns= None)
  f = open('/content/drive/MyDrive/Text/encode', 'w')
  f.write(ascii)
  f.close()

DNA write(y/n) : y
